<a href="https://colab.research.google.com/github/CesarChalcoElias/prompting-techniques/blob/main/01_CoT_prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chain of Thought (CoT) Prompting

### **Overview**  
This tutorial introduces Chain of Thought (CoT) prompting, a powerful technique in prompt engineering that encourages AI models to break down complex problems into step-by-step reasoning processes. We'll explore how to implement CoT prompting using OpenAI's GPT models and the LangChain library.

### **Motivation**  
As AI language models become more advanced, there's an increasing need to guide them towards producing more transparent, logical, and verifiable outputs. CoT prompting addresses this need by encouraging models to show their work, much like how humans approach complex problem-solving tasks. This technique not only improves the accuracy of AI responses but also makes them more interpretable and trustworthy.

## Setup

In [1]:
!pip install langchain-openai -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/84.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 38.5 MB/s eta 0:00:00


In [6]:
import os
from google.colab import userdata

from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

In [7]:
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [8]:
llm = ChatOpenAI(model_name="gpt-5-nano")

## Basic CoT Prompting

In [9]:
# Standard prompt
standard_prompt = PromptTemplate(
    input_variables=["question"],
    template="Answer the following question concisely: {question}."
)

In [10]:
# Chain of Thought prompt
cot_prompt = PromptTemplate(
    input_variables=["question"],
    template="Answer the following question step by step concisely: {question}"
)

In [11]:
# Create chains
standard_chain = standard_prompt | llm
cot_chain = cot_prompt | llm

In [12]:
# Example question
question = "If a train travels 120 km in 2 hours, what is its average speed in km/h? assuming uniform rectilinear movement"

In [13]:
# Get responses
standard_response = standard_chain.invoke(question).content
cot_response = cot_chain.invoke(question).content

print("Standard Response:")
print(standard_response)
print("=" * 40)
print("\nChain of Thought Response:")
print(cot_response)

Standard Response:
60 km/h.

Chain of Thought Response:
Step 1: Given distance d = 120 km and time t = 2 h.  
Step 2: For uniform rectilinear motion, average speed v = d/t.  
Step 3: Compute v = 120 km / 2 h = 60 km/h.  

Answer: 60 km/h.


## Advanced CoT techniques

In [14]:
# Advanced prompt template for CoT
advanced_cot_prompt = PromptTemplate(
    input_variables=["question"],
    template="""Solve the following problem step by step. For each step:
1. State what you're going to calculate
2. Write the formula you'll use (if applicable)
3. Perform the calculation
4. Explain the result

Question: {question}

Solution:"""
)

In [15]:
# Create the chain with the new complex prompt template
advanced_cot_chain = advanced_cot_prompt | llm

In [16]:
complex_question = "A car travels 150 km at 60 km/h, then another 100 km at 50 km/h. What is the average speed for the entire journey?"

In [17]:
advanced_cot_response = advanced_cot_chain.invoke(complex_question).content

print("Advanced CoT Response:")
print(advanced_cot_response)

Advanced CoT Response:
Step 1: What I'm going to calculate
- Calculate the time taken for the first leg of the trip.
- Formula: t1 = d1 / v1
- Calculation: t1 = 150 km / 60 km/h = 2.5 h
- Explanation: Traveling 150 km at 60 km/h takes 2.5 hours.

Step 2: What I'm going to calculate
- Calculate the time taken for the second leg of the trip.
- Formula: t2 = d2 / v2
- Calculation: t2 = 100 km / 50 km/h = 2 h
- Explanation: Traveling 100 km at 50 km/h takes 2 hours.

Step 3: What I'm going to calculate
- Compute total distance and total time for the entire journey.
- Formulas: D_total = d1 + d2; T_total = t1 + t2
- Calculation: D_total = 150 + 100 = 250 km; T_total = 2.5 + 2 = 4.5 h
- Explanation: The whole trip covers 250 km and takes 4.5 hours.

Step 4: What I'm going to calculate
- Determine the average speed for the entire journey.
- Formula: v_avg = D_total / T_total
- Calculation: v_avg = 250 km / 4.5 h ≈ 55.56 km/h
- Explanation: Average speed is total distance divided by total time

## Comparative Analysis

In [18]:
# Let's try with a more challenging question
challenging_question = """
A cylindrical water tank with a radius of 1.5 meters and a height of 4 meters is 2/3 full.
If water is being added at a rate of 10 liters per minute, how long will it take for the tank to overflow?
Give your answer in hours and minutes, rounded to the nearest minute.
(Use 3.14159 for π and 1000 liters = 1 cubic meter)"""

standard_response = standard_chain.invoke(challenging_question).content
cot_response = advanced_cot_chain.invoke(challenging_question).content

print("Standard Response:")
print(standard_response)
print("\nChain of Thought Response:")
print(cot_response)

Standard Response:
Volume capacity: V = πr^2h = π(1.5)^2(4) = 9π ≈ 28.274 m³ = 28,274 L.

Current water (2/3 full): (2/3) of 28,274 L ≈ 18,850 L.
Remaining to full: 28,274 − 18,850 ≈ 9,424 L ≈ 9,424.77 L.

Fill rate = 10 L/min ⇒ time = 9,424.77 / 10 ≈ 942.48 min ≈ 15 h 42 min.

Answer: 15 hours 42 minutes.

Chain of Thought Response:
Step 1: Calculate the full capacity of the tank.
- What I’m going to calculate: The total volume when the tank is full.
- Formula: V = π r^2 h
- Calculation: V = 3.14159 × (1.5)^2 × 4 = 3.14159 × 2.25 × 4 = 3.14159 × 9 = 28.27431 m^3
- Explanation: The tank’s full capacity is 28.27431 cubic meters.

Step 2: Determine the current water volume (tank is 2/3 full).
- What I’m going to calculate: Current water volume V0.
- Formula: V0 = (2/3) × V
- Calculation: V0 = (2/3) × 28.27431 = 18.84954 m^3
- Explanation: When 2/3 full, there are 18.84954 m^3 of water in the tank.

Step 3: Compute the additional volume needed to reach overflow.
- What I’m going to calcul

## Problem-solving scenario

In [19]:
logical_reasoning_prompt = PromptTemplate(
    input_variables=["scenario"],
    template="""Analyze the following logical puzzle thoroughly. Follow these steps in your analysis:

List the Facts:

Summarize all the given information and statements clearly.
Identify all the characters or elements involved.
Identify Possible Roles or Conditions:

Determine all possible roles, behaviors, or states applicable to the characters or elements (e.g., truth-teller, liar, alternator).
Note the Constraints:

Outline any rules, constraints, or relationships specified in the puzzle.
Generate Possible Scenarios:

Systematically consider all possible combinations of roles or conditions for the characters or elements.
Ensure that all permutations are accounted for.
Test Each Scenario:

For each possible scenario:
Assume the roles or conditions you've assigned.
Analyze each statement based on these assumptions.
Check for consistency or contradictions within the scenario.
Eliminate Inconsistent Scenarios:

Discard any scenarios that lead to contradictions or violate the constraints.
Keep track of the reasoning for eliminating each scenario.
Conclude the Solution:

Identify the scenario(s) that remain consistent after testing.
Summarize the findings.
Provide a Clear Answer:

State definitively the role or condition of each character or element.
Explain why this is the only possible solution based on your analysis.
Scenario:

{scenario}

Analysis:""")

logical_reasoning_chain = logical_reasoning_prompt | llm

logical_puzzle = """In a room, there are three people: Amy, Bob, and Charlie.
One of them always tells the truth, one always lies, and one alternates between truth and lies.
Amy says, 'Bob is a liar.'
Bob says, 'Charlie alternates between truth and lies.'
Charlie says, 'Amy and I are both liars.'
Determine the nature (truth-teller, liar, or alternator) of each person."""

logical_reasoning_response = logical_reasoning_chain.invoke(logical_puzzle).content
print(logical_reasoning_response)

Analysis

1) List the Facts
- There are three people: Amy, Bob, and Charlie.
- Roles: one is a truth-teller (always tells the truth), one is a liar (always lies), and one is an alternator (sometimes tells the truth, sometimes lies; alternates between the two across statements).
- Statements:
  - Amy: "Bob is a liar."
  - Bob: "Charlie alternates between truth and lies."
  - Charlie: "Amy and I are both liars."

2) Identify all the characters
- Amy
- Bob
- Charlie

3) Identify Possible Roles or Conditions
- The three roles to assign: Truth-teller (T), Liar (L), Alternator (A).
- Since there are three people and three roles, each role must be held by a different person.
- Important note on alternator: with only one statement per person in this puzzle, an alternator’s single statement could be either truth or lie. However, the content of each statement must be consistent with the person’s assigned role and with the other two statements.

4) Note the Constraints
- Exactly one truth-teller,

## Conclusions

My conclusions are:
* We can actually improve our LLM responses jus by asking the model to show the process step by step.
* In complex scenarios it's helpful to not just ask to "answer step by step" but being more specific on each step specifying what we want to consider, to do, to analyze, to avoid and to return. This makes even more robust the reasoning of the model.